# Домашнее задание №7: ДЗ7-бонусное


### Зафиксируйте какую-либо многомерную скалярную функцию и реализуйте для неё анализ чувствительности по методу Соболя на Python с использованием библиотеки SALib.

In [1]:
import time
import numpy as np
from numba import njit, prange
from SALib.sample import saltelli
from SALib.analyze import sobol

### Проведите анализ чувствительности, проверьте сходимость, измерьте тайминги, 25 баллов 

In [2]:
a = 0.5
b = 0.8

In [3]:
def evaluate_model(x):
    return np.cos(x[0]) + a * np.sin(x[2]) * np.cos(x[2]) + b * x[0]**3 * np.cos(x[1])**3

In [4]:
problem = {
    'num_vars' : 3,
    'names' : ['x1', 'x2', 'x3'],
    'bounds' : [[-3, 3],
                [-3, 3],
                [-3, 3]
                ]
}

In [5]:
n = 100000
start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = evaluate_model(X)

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))
# first-order indices
print(Si['S1'])
# total indices for each variable
print(Si['ST'])
# second order indices
print("x1-x2:", Si['S2'][0,1])
print("x1-x3:", Si['S2'][0,2])
print("x2-x3:", Si['S2'][1,2])

/var/folders/b7/t1bwy2bj38s0vjf_n9kzhzjw0000gq/T/ipykernel_76364/2900001500.py:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
/opt/homebrew/Caskroom/miniforge/base/envs/env/lib/python3.8/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (100000) is equal to `2^n`.
        
  warnings.warn(msg)


samples generation took 3.2918219566345215 seconds
model evaluation took 3.3362138271331787 seconds
SA took 4.213747024536133 seconds
[0.03219263 0.00018097 0.0017024 ]
[0.99873225 0.96579416 0.00169775]
x1-x2: 0.9656643718339771
x1-x3: 0.00041588620423737167
x2-x3: -0.0004194848737831959


### Ускорьте вычисления Python с использованием любой из имеющихся возможностей (PyBind11, ctypes, cython, numba), 25 баллов

In [6]:
@njit
def opt_evaluate_model(x):
    return np.cos(x[0]) + a * np.sin(x[2]) * np.cos(x[2]) + np.sin(x[1])**2 + b * x[0]**3 * np.cos(x[1])**3

In [7]:
n = 100000
start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = opt_evaluate_model(X)

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))

/var/folders/b7/t1bwy2bj38s0vjf_n9kzhzjw0000gq/T/ipykernel_76364/1940001659.py:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)


samples generation took 3.292879104614258 seconds
model evaluation took 0.5063209533691406 seconds
SA took 4.137076139450073 seconds


### Попробуйте добавить параллелизм в вычисления, 25 баллов  

In [8]:
n = 100000
start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i in prange(n):
    Y[i] = opt_evaluate_model(param_values[i])

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))

/var/folders/b7/t1bwy2bj38s0vjf_n9kzhzjw0000gq/T/ipykernel_76364/3006892186.py:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
/opt/homebrew/Caskroom/miniforge/base/envs/env/lib/python3.8/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (100000) is equal to `2^n`.
        
  warnings.warn(msg)


samples generation took 3.316559076309204 seconds
model evaluation took 0.03644299507141113 seconds
SA took 4.184094190597534 seconds
